In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os
from tqdm import tqdm

# ------------------ Configs ------------------ #
data_dir = '/kaggle/input/ip02-dataset/classification/'  # should contain train/, val/, test/ folders
batch_size = 64
learning_rate = 0.01
num_epochs = 30
lr_step_size = 40
lr_gamma = 0.1
momentum = 0.9
weight_decay = 0.0005
dropout_rate = 0.3
input_size = 224
num_workers = 4
# num_classes will be determined from the dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ------------------ Data Transforms ------------------ #
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

# ------------------ Datasets & Dataloaders ------------------ #
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transforms[x])
    for x in ['train', 'val', 'test']
}

num_classes = len(image_datasets['train'].classes)
print("✅ num_classes set to:", num_classes)

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=(x != 'test'), num_workers=num_workers)
    for x in ['train', 'val', 'test']
}

class_names = image_datasets['train'].classes

# ------------------ Model Setup ------------------ #
model = models.resnet50(pretrained=True)
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=dropout_rate),
    nn.Linear(in_features, num_classes)
)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum, weight_decay=weight_decay)
scheduler = StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)

# ------------------ Training Loop ------------------ #
def train_model():
    best_val_acc = 0.0
    best_model_path = 'best_resnet50_model.pth'

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 20)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # 🔽 Save best model on validation
            if phase == 'val' and epoch_acc > best_val_acc:
                best_val_acc = epoch_acc
                torch.save(model.state_dict(), best_model_path)
                print(f"✅ Best model saved at epoch {epoch+1} with val acc: {epoch_acc:.4f}")

        scheduler.step()

    print(f"\n🏁 Training completed. Best Val Acc: {best_val_acc:.4f}")


# ------------------ Evaluation on Test Set ------------------ #
def evaluate_model():
    model.eval()
    all_preds = []
    all_labels = []
    top1_correct = 0
    top5_correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(dataloaders['test'], desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, top1_preds = outputs.topk(1, dim=1)
            top5_preds = outputs.topk(5, dim=1).indices

            all_preds.extend(top1_preds.squeeze().cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            top1_correct += (top1_preds.squeeze() == labels).sum().item()

            for i in range(labels.size(0)):
                if labels[i] in top5_preds[i]:
                    top5_correct += 1

            total += labels.size(0)

    top1_acc = top1_correct / total
    top5_acc = top5_correct / total

    print(f"\n✅ Top-1 Accuracy: {top1_acc:.4f}")
    print(f"✅ Top-5 Accuracy: {top5_acc:.4f}\n")

    print("📊 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    print("🧾 Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

# ------------------ Run ------------------ #
if __name__ == "__main__":
    train_model()
    evaluate_model()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


✅ num_classes set to: 102


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 269MB/s]


Epoch 1/30
--------------------
train Loss: 2.2041 Acc: 0.4515
val Loss: 1.8279 Acc: 0.5197
✅ Best model saved at epoch 1 with val acc: 0.5197
Epoch 2/30
--------------------
train Loss: 1.5644 Acc: 0.5749
val Loss: 1.6121 Acc: 0.5619
✅ Best model saved at epoch 2 with val acc: 0.5619
Epoch 3/30
--------------------
train Loss: 1.3391 Acc: 0.6270
val Loss: 1.5899 Acc: 0.5790
✅ Best model saved at epoch 3 with val acc: 0.5790
Epoch 4/30
--------------------
train Loss: 1.1778 Acc: 0.6635
val Loss: 1.5275 Acc: 0.5903
✅ Best model saved at epoch 4 with val acc: 0.5903
Epoch 5/30
--------------------
train Loss: 1.0612 Acc: 0.6896
val Loss: 1.6234 Acc: 0.5707
Epoch 6/30
--------------------
train Loss: 0.9688 Acc: 0.7143
val Loss: 1.7634 Acc: 0.5478
Epoch 7/30
--------------------
train Loss: 0.8783 Acc: 0.7392
val Loss: 1.5081 Acc: 0.6067
✅ Best model saved at epoch 7 with val acc: 0.6067
Epoch 8/30
--------------------
train Loss: 0.8270 Acc: 0.7518
val Loss: 1.6930 Acc: 0.5775
Epoch 9/3

Evaluating: 100%|██████████| 354/354 [01:22<00:00,  4.29it/s]



✅ Top-1 Accuracy: 0.5803
✅ Top-5 Accuracy: 0.8188

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.66      0.64       335
           1       0.27      0.32      0.30       147
          10       0.53      0.64      0.58       257
         100       0.48      0.56      0.52       138
         101       0.79      0.72      0.75      1723
          11       0.26      0.42      0.32       122
          12       0.79      0.42      0.55        52
          13       0.25      0.37      0.30       123
          14       0.78      0.73      0.75       258
          15       0.97      0.84      0.90       495
          16       0.63      0.68      0.65       267
          17       0.52      0.33      0.41        45
          18       0.47      0.39      0.42       257
          19       0.30      0.19      0.23       148
           2       0.34      0.30      0.32        79
          20       0.41      0.22      0.28       144
    